In [1]:
import multiprocessing as mp
import GPUtil
import platform
import psutil
import cupy as cp

# ---------------------------
# SYSTEM INFO
# ---------------------------
def show_system_info():
    print("\n=== SYSTEM INFO ===")
    print(f"CPU cores: {mp.cpu_count()}")
    print(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
    print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")
    print(f"CPU usage: {psutil.cpu_percent():.1f}%")
    print(f"CPU model: {platform.processor()}")
    print(f"OS: {platform.system()} {platform.release()}")

    # GPU detection
    gpu_info_str = "None"
    # Try NVIDIA CUDA
    try:
        
        cp.cuda.runtime.getDevice()
        gpu_name = cp.cuda.runtime.getDeviceProperties(0)['name'].decode()
        gpu_info_str = f"NVIDIA GPU: {gpu_name}"
    except (ImportError, cp.cuda.runtime.CUDARuntimeError):
        # Fallback to GPUtil
        try:
            
            gpus = GPUtil.getGPUs()
            if gpus:
                gpu_info_str = f"{gpus[0].name} (non-CUDA GPU)"
            else:
                gpu_info_str = "No GPU detected"
        except ImportError:
            gpu_info_str = "GPU detection skipped (GPUtil not installed)"

    print(f"GPU: {gpu_info_str}")

print("=== 🔧 SYSTEM PERFORMANCE BENCHMARK ===")
show_system_info()

ModuleNotFoundError: No module named 'GPUtil'

In [2]:
! conda install -y -c conda-forge GPUtil

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/etl

  added / updated specs:
    - gputil


The following NEW packages will be INSTALLED:

  gputil             conda-forge/noarch::gputil-1.4.0-pyhd8ed1ab_1 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [ ]:
import time
import psutil
# import timeit

def cpu_intensive_task():
    # Example: Perform a large number of calculations
    result = 0
    for i in range(10**8 * 3):
        result += i * i

def measure_cpu_usage_during(func, iterations):
    cpu_readings = []
    start_time = time.time()

    # Start a background measurement loop
    for _ in range(iterations):
        # Start measuring before each iteration
        cpu_before = psutil.cpu_percent(interval=None)

        # Run the function
        func()

        # Measure after this iteration
        cpu_after = psutil.cpu_percent(interval=None)
        cpu_readings.append(cpu_after)

    end_time = time.time()
    duration = end_time - start_time

    avg_cpu = sum(cpu_readings) / len(cpu_readings)
    return duration / iterations, avg_cpu

if __name__ == "__main__":
    iterations = 10

    avg_time, avg_cpu = measure_cpu_usage_during(cpu_intensive_task, iterations)
    print(f"✅ Average CPU task time: {avg_time:.4f} seconds per run")
    print(f"📊 Average CPU usage: {avg_cpu:.2f}%")

✅ Average CPU task time: 13.8901 seconds per run
📊 Average CPU usage: 17.64%


In [ ]:
from multiprocessing import Pool, cpu_count
import time

def stress_cpu(iterations):
    for _ in range(iterations):
        _ = 1 + 1  # Simple operation to keep CPU busy

if __name__ == '__main__':
    num_processes = cpu_count()
    print(f"Stress testing {num_processes} CPU cores...")
    with Pool(num_processes) as pool:
        pool.map(stress_cpu, [10**7] * num_processes)
    print("CPU stress test complete.")

Stress testing 14 CPU cores...


In [15]:
import psutil
import time

print(f"Initial RAM usage: {psutil.virtual_memory().percent}%")

# Allocate a large list to consume memory
large_list = [i for i in range(10**8*3)]
time.sleep(1) # Allow system to update memory stats

print(f"RAM usage after allocation: {psutil.virtual_memory().percent}%")
del large_list # Release memory
time.sleep(1)

print(f"RAM usage after deallocation: {psutil.virtual_memory().percent}%")

Initial RAM usage: 44.4%
RAM usage after allocation: 78.4%
RAM usage after deallocation: 42.0%


In [16]:
# Example using CuPy for GPU operations
import cupy as cp
import time

try:
    # Create a large array on the GPU
    gpu_array = cp.random.rand(10000, 10000)

    start_time = time.time()
    # Perform a matrix multiplication on the GPU
    result = cp.dot(gpu_array, gpu_array)
    end_time = time.time()

    print(f"GPU matrix multiplication time: {end_time - start_time:.4f} seconds")
except Exception as e:
    print(f"{e}")
except cp.cuda.runtime.CudaAPIError:
    print("CuPy or CUDA not detected. Ensure a compatible GPU and drivers are installed.")

cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version
